In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Online Synthetic (OS) 方法实现
- 根据当前问题动态生成多个输入-输出样例
- 作为Prompt输入来指导SQL生成
"""

import json
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple
import torch
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
)
import re

# ---------- 可调参数 ----------
@dataclass
class Config:
    model_name: str = r"/home/yangliu26/qwen3-8b"  
    input_json: str = r"/home/yangliu26/data/train/schema_linking_result.json"
    output_dir: str = r"/home/yangliu26/CHASE/candidates/os_results"
    # 文本生成超参
    max_new_tokens: int = 1024
    do_sample: bool = True
    temperature: float = 0.7
    # 性能设置
    batch_size: int = 4
    use_fp16: bool = True
    device_map: str = "auto"
    # OS特定参数
    num_general_examples: int = 3
    num_schema_aware_examples: int = 3

# 配置实例
CFG = Config()

In [3]:
def load_model_and_tokenizer(cfg: Config):
    """加载模型和分词器"""
    # 量化配置
    quant_cfg = None
    if not cfg.use_fp16:
        quant_cfg = BitsAndBytesConfig(load_in_8bit=True)

    tokenizer = AutoTokenizer.from_pretrained(
        cfg.model_name,
        trust_remote_code=True,
        padding_side="left",
    )
    model = AutoModelForCausalLM.from_pretrained(
        cfg.model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16 if cfg.use_fp16 else torch.float32,
        quantization_config=quant_cfg,
        device_map=cfg.device_map,
    )
    return tokenizer, model
tokenizer, model = load_model_and_tokenizer(CFG)

/opt/conda/envs/envd/lib/python3.10/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
def load_prompt_template(path: str) -> str:
    with open(path, encoding="utf-8") as f:
        return f.read()



def generate_examples_by_schema(db_schema: str, relevant_columns: List[str], num_examples: int, generator) -> List[Tuple[str, str]]:
    SCHEMA_AWARE_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/schema_aware_template.txt")
    """生成基于特定schema的示例"""
    prompt = SCHEMA_AWARE_TEMPLATE.format(
        db_schema=db_schema,
        relevant_columns='\n'.join(relevant_columns),
        num_examples=num_examples
    )
    
    response = generator(prompt, max_new_tokens=2048, do_sample=True, temperature=0.8)
    text = response[0]["generated_text"].strip()
    
    # 解析示例
    examples = []
    lines = text.split('\n')
    i = 0
    while i < len(lines):
        if lines[i].startswith("Question:"):
            question = lines[i][3:].strip()
            i += 1
            # 寻找SQL行
            while i < len(lines) and not lines[i].startswith("SQL:"):
                i += 1
            if i < len(lines):
                sql = lines[i][4:].strip()
                examples.append((question, sql))
        i += 1
    
    return examples[:num_examples]  # 确保不超过请求的数量

def format_few_shot_prompt(examples: List[Tuple[str, str]], question: str, db_schema: str) -> str:
    FEW_SHOT_TEMPLATE = load_prompt_template("/home/yangliu26/CHASE/template/os_few_shot_template.txt")
    """格式化few-shot提示"""
    examples_text = ""
    for i, (q, sql) in enumerate(examples, 1):
        examples_text += f"示例 {i}:\n问题: {q}\nSQL: {sql}\n\n"
    
    return FEW_SHOT_TEMPLATE.format(
        db_schema=db_schema,
        examples=examples_text,
        question=question
    )

def extract_sql(text: str) -> str:
    """从生成的文本中提取SQL"""
    # 尝试找到SQL:后面的内容
    if "SQL:" in text:
        return text.split("SQL:", 1)[1].strip()
    return text.strip()

def online_synthetic_icl(question: str, db_schema: str, generator, 
                         num_general: int = 3, num_schema_aware: int = 3):
    """主函数：使用在线合成示例方法生成SQL"""
    # 步骤1: 使用常见SQL特征生成通用示例
    general_examples = generate_examples_by_sql_features(db_schema, num_general, generator)
    
    # 步骤2: 使用过滤列生成schema-aware示例
    relevant_columns = filter_columns_by_question(question, db_schema, generator)
    schema_examples = generate_examples_by_schema(db_schema, relevant_columns, num_schema_aware, generator)
    
    # 步骤3: 组合所有示例 + 当前问题进入Prompt
    prompt = format_few_shot_prompt(general_examples + schema_examples, question, db_schema)
    
    # 步骤4: 生成SQL
    response = generator(prompt, max_new_tokens=1024, do_sample=False)
    return extract_sql(response[0]["generated_text"])

In [8]:
# 加载数据
with open(CFG.input_json, 'r', encoding='utf-8') as f:
    data = json.load(f)
len(data)

10

In [9]:
generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
    )

Device set to use cpu


In [11]:
item = data[0]
question = item.get("question", "")
db_schema = item.get("schema_linking", "")
question, db_schema

('Name movie titles released in year 1945. Sort the listing by the descending order of movie popularity.',
 {'movies': ['movie_title',
   'movies',
   'movie_title_language',
   'movie_release_year',
   'movie_popularity',
   'movie_release_year',
   'movie_title',
   'movies',
   'movie_id',
   'movie_release_year',
   'movies',
   'movie_title',
   'movie_popularity',
   'movies',
   'movie_title',
   'movie_popularity',
   'movies'],
  'ratings': ['movie_id',
   'critic',
   'ratings',
   'critic',
   'ratings',
   'critic_likes',
   'rating_url'],
  'lists': ['lists']})

In [ ]:
generate_examples_by_sql_features(

In [14]:
def generate_examples_by_sql_features(db_schema: str, num_examples: int, generator) -> List[Tuple[str, str]]:
    # GENERAL_EXAMPLES_TEMPLATE是生成通用示例的提示模板
    GENERAL_EXAMPLES_TEMPLATE = load_prompt_template(r"/home/yangliu26/CHASE/template/general_examples_template.txt")
    """生成涵盖不同SQL特性的示例"""
    prompt = GENERAL_EXAMPLES_TEMPLATE.format(
        db_schema=db_schema,
        num_examples=num_examples
    )
    
    response = generator(prompt, max_new_tokens=2048, do_sample=True, temperature=0.8)
    text = response[0]["generated_text"].strip()
    
    # 解析示例
    examples = []
    lines = text.split('\n')
    i = 0
    while i < len(lines):
        if lines[i].startswith("问题:"):
            question = lines[i][3:].strip()
            i += 1
            # 寻找SQL行
            while i < len(lines) and not lines[i].startswith("SQL:"):
                i += 1
            if i < len(lines):
                sql = lines[i][4:].strip()
                examples.append((question, sql))
        i += 1
    return examples[:num_examples]  # 确保不超过请求的数量
# generate_examples_by_sql_features(db_schema, 3, generator)

In [21]:
GENERAL_EXAMPLES_TEMPLATE = load_prompt_template(r"/home/yangliu26/CHASE/template/general_examples_template.txt")
prompt = GENERAL_EXAMPLES_TEMPLATE.format(
        db_schema=db_schema,
        num_examples=3
    )
print(prompt)

You are a professional database expert. Your task is to generate natural-language questions and corresponding SQL query examples for the given database.

**Database information:**
{'movies': ['movie_title', 'movies', 'movie_title_language', 'movie_release_year', 'movie_popularity', 'movie_release_year', 'movie_title', 'movies', 'movie_id', 'movie_release_year', 'movies', 'movie_title', 'movie_popularity', 'movies', 'movie_title', 'movie_popularity', 'movies'], 'ratings': ['movie_id', 'critic', 'ratings', 'critic', 'ratings', 'critic_likes', 'rating_url'], 'lists': ['lists']}

Please create **3** distinct pairs of natural-language questions and SQL query examples. The examples should demonstrate a range of SQL features, including:

1. A simple **SELECT** query
2. A query with a **WHERE** clause
3. A multi-table **JOIN** query
4. A query that uses **GROUP BY** with aggregate functions
5. A query with **ORDER BY**
6. A query with **LIMIT**
7. A complex query that contains a **subquery**

